In [2]:
from geopy.geocoders import Nominatim
from geopy import distance
import pandas as pd
import numpy as np
import math
import re
from Levenshtein import *

In [3]:
excel = 'transmission.xlsx'
sheet = 'Trans.Investments'

df = pd.read_excel(excel, sheet_name=sheet)

# TODO: might not work in later versions, use sanity check.
df.columns = df.iloc[0]
df         = df.drop(index=df.index[0])

# TODO: also contains new substations!

wanted_columns = ['Investment number',
                  'This investment belongs to project number…',
                  'Commissioning Year',
                  'Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction',
                  'Type of Element',
                  'Substation From',
                  'Substation To',
                  'Technology',
                  'Total route length (km)']
df[wanted_columns]

,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km)
1,2,1,2024,2,ACTransmissionLine,Pedralva (PT),Sobrado (PT),AC,67
2,4,1,2022,3,ACTransmissionLine,V.Minho (by Ribeira de Pena),Feira (by Ribeira de Pena),AC,131
3,474,1,2021,3,OnshoreSubstation,Ribeira de Pena (PT),-,AC,0
4,18,4,2022,3,ACTransmissionLine,Beariz (ES),Fontefria (ES),AC,30
5,496,4,2022,3,ACTransmissionLine,Fontefria (ES),Vila Nova de Famalicão (PT) (By Ponte de Lima),AC,140.21
...,...,...,...,...,...,...,...,...,...
317,1747,1077,2036,1,DCTransmissionLine,Thessaloniki,Dubrovo,DC,110
318,1748,1077,2036,1,DCTransmissionLine,Dubrovo,Ch. Mogila,DC,170
319,1745,1081,2036,1,OffshoreDCTransmissionCable,Tobruk,Arachtos,DC,1070
320,1749,1081,2036,1,DCTransmissionLine,Arachtos,Elbasan,DC,180


In [213]:
wanted   = df[wanted_columns]
ac_lines = wanted.loc[wanted['Type of Element'] == 'ACTransmissionLine']
# ac_lines = ac_lines.reindex(columns=ac_lines.columns.tolist() + ['x1', 'y1', 'x2', 'y2', 'coord_distance'])
# ac_lines = ac_lines.loc[]

In [51]:
ac_lines.loc[ac_lines['Status ID%'] >= 3]

KeyError: 'Status ID%'

# Use bus names from buses.csv (v0.1.0)

In [88]:
buses_file = 'buses_v0.1.0.csv'

# see base_network.py in PyPSA-Eur repository
buses = (pd.read_csv(buses_file, quotechar="'",
                     true_values='t', false_values='f',
                     dtype=dict(bus_id="str"))
        .set_index("bus_id")
        .drop(['station_id'], axis=1)
        .rename(columns=dict(voltage='v_nom')))

In [89]:
no_tags = buses[buses['tags'].isna()].index.size
print(f'{no_tags} buses have no tags.')

yes_tags = buses[~buses['tags'].isna()].index.size
print(f'{yes_tags} buses have tags.')

1238 buses have no tags.
6773 buses have tags.


In [90]:
buses = buses.loc[~buses['tags'].isna()]
buses = buses.loc[buses.symbol == 'Substation']

# Extract 'name_eng' and 'country' from tags in  buses

In [146]:
split_regex = r'("\w+"=>"[^"]*"),'

tag_regex   = r'"(?P<key>\w+)"=>"(?P<value>[^"]*)"'
tag_pattern = re.compile(tag_regex)

rows = []

for index, row in buses.iterrows():
    name    = ''
    country = ''
    x = row['x']
    y = row['y']
    
    tags_string = row['tags']
    
    tags = re.split(split_regex, tags_string)
    
    # Remove whitespaces at front and end, remove None values
    tags = [s.strip() for s in tags]
    tags = list(filter(None, tags))
    
    for tag in tags:
        m = tag_pattern.match(tag)
            
        if m is None:
            print(tag)
            
        # see group names in tag_regex
        key   = m.group('key')
        value = m.group('value')
        
        if key == 'name_eng':
            name = value.strip()
        elif key == 'country':
            country = value.strip()
    
    if name == 'unknown' or not name:
        continue
        
    rows.append((name, country, x, y))

In [147]:
curated_buses = pd.DataFrame.from_records(rows, columns=['name', 'country', 'x', 'y'])

## Remove duplicate rows

In [148]:
curated_buses = curated_buses.loc[~curated_buses.duplicated()]

In [189]:
# curated_buses.loc[curated_buses.name == 'Jeddah'].country

curated_buses.loc[curated_buses.name == 'Shoaibah'].iloc[0]['x']

39.726562

## There are substations which share the same name but have different coordinates
- large deviation between coordinates => substations are most likely in different countries 
    - BUT: it does occur that different places in the same country get the same name
- small deviation between coordinates => reference to same substation (error in gridextract?)

### List of all duplicates

In [175]:
duplicated = curated_buses.loc[curated_buses.name.duplicated()]

for name in duplicated.name.unique():
    print(name)
    for index, row in curated_buses.loc[curated_buses['name'] == name].iterrows():
        print(f"({row['x']}, {row['y']}), {row['country']}")
    print('----')

Jeddah
(39.716949, 21.985075), SA
(39.711456, 22.038549), SA
----
Al Madina
(39.635925, 24.775513), SA
(39.508209, 24.718143), SA
----
I. Baroud
(30.734253, 30.955236), EG
(30.890808, 30.855079), EG
----
Local
(25.041962, 31.598422), LY
(24.134216, 31.960318), LY
----
Sahab
(36.021423, 31.809895), JO
(36.478729, 32.261588), JO
----
Tazoult
(6.378937, 35.424868), DZ
(6.49154699999999, 35.373375), DZ
----
Saida
(35.400696, 33.587167), LB
(0.146942000000011, 34.908458), DZ
----
Tizi Ouzou
(4.15557899999999, 36.698154), DZ
(4.000397, 36.561497), DZ
----
Jerada
(-2.06405600000001, 33.876117), MA
(-2.11898799999999, 34.335498), MA
----
Taza
(-4.139099, 34.271971), MA
(-3.95919799999999, 34.268566), MA
----
Naama 400
(-0.365294999999999, 33.26625), DZ
(-0.429840000000002, 33.366091), DZ
----
Brindisi /S.
(17.992859, 40.571197), IT
(17.862396, 40.520063), IT
----
Tunis
(10.140381, 36.73118), TN
(10.055237, 36.79499), TN
----
PB 1
(7.04772899999999, 36.857648), DZ
(7.138367, 36.891703), DZ
----

### Same name and country, large deviations

In [172]:
curated_buses.loc[curated_buses['name'] == 'Yuzhnaya']

,name,country,x,y
2720,Yuzhnaya,RU,44.817352,48.155093
3851,Yuzhnaya,RU,50.674438,52.002638
3905,Yuzhnaya,RU,36.268616,51.642737
3927,Yuzhnaya,RU,38.685608,51.843414
5378,Yuzhnaya,RU,59.824677,56.576128


### Same name, different country, large deviation

In [169]:
curated_buses.loc[curated_buses.name == 'Saida']

,name,country,x,y
617,Saida,LB,35.400696,33.587167
833,Saida,DZ,0.146942,34.908458


In [170]:
curated_buses.loc[curated_buses.name == 'Titan']

,name,country,x,y
1986,Titan,AL,19.786377,41.619549
2825,Titan,UA,33.767853,46.195993
5825,Titan,RU,34.026031,67.451763


## create mapping from all unique tyndp substation names to substation names from 'buses'

In [214]:
tyndp_subs   = set(ac_lines['Substation From']).union(set(ac_lines['Substation To']))
tyndp_to_bus = {}

for tyndp in tyndp_subs:
    buses_subs = curated_buses.name.values
    
    closest = min([(bus, distance(bus, tyndp)) for bus in buses_subs], key=lambda t: t[1])[0]
    
    tyndp_to_bus[tyndp] = closest

In [174]:
tyndp_to_bus

{'SS Sándorfalva': 'Sándorfalva',
 'Altomonte (IT)': 'Altomonte',
 'Creys (FR)': 'Creys',
 'Djerdap': 'Djerdap 1',
 'Oldstreet 400kV': 'Oldstreet',
 'Ekhyddan (SE)': 'Ekudden',
 'Zell-Ziller (AT)': 'Zell/Ziller',
 'Visegrad (BA)': 'Visegrad',
 'Konjsko (HR)': 'Konjsko',
 'Conneforde (DE)': 'Conneforde',
 'Sajoivanka (HU)': 'Sajóivánka',
 'Massenhoven': 'Massenhoven',
 'Kilingi-Nomme (EE)': 'Kilingi-Nomme',
 'Darbenai': 'Darna',
 'Chamoson': 'Chamoson',
 'Tsirguliina ': 'Tsirguliina',
 'Bajina Basta (RS)': 'Bajina Basta',
 'Center (TBD)': 'Centenario',
 'Vilnius': 'Vilnius',
 'Beariz (ES)': 'Beariz',
 'SS Subotica 3': 'Subotica',
 'SS Srbobran': 'Srbobran',
 'Wahle (DE)': 'Wahle',
 'Cernavoda (RO)': 'Craiova Nord',
 'Baczyna': 'Baczyna',
 'Montalto di Castro (IT) Terna station 400 kV AC': 'Montezic poste et centrale',
 'Razbojiste': 'La Boisse',
 'Resita (RO)': 'Relizane',
 'Wullenstetten (DE)': 'Eichstetten',
 'Ladce (SK)': 'Lac Noir',
 'Klixbüll (DE)': 'Tripoli SE',
 'Woodland (IE)': 

In [231]:
# Several matches in different countries -> try to extract country code from tyndp name
fr_to_tuples = {}

for index, row in ac_lines.iterrows():
    # TODO: improve this. For now, we take the first occurence and hope for the best.
    
    # Get closest name match based on Levenshtein distance for start- and endpoints of line
    s1 = tyndp_to_bus[row['Substation From']]
    s2 = tyndp_to_bus[row['Substation To']]
    
    # Extract respective rows in buses to determine coordinates
    buses_s1_row = curated_buses.loc[curated_buses.name == s1].iloc[0]
    buses_s2_row = curated_buses.loc[curated_buses.name == s2].iloc[0]
            
    x1 = buses_s1_row['x']
    y1 = buses_s1_row['y']
    
    x2 = buses_s2_row['x']
    y2 = buses_s2_row['y']
    
    # TODO: determine distance of 2 points. If higher than given line length, discard as error row.
    
    fr_to_tuples[index] = (x1, y1, x2, y2)

In [232]:
coordinates = pd.DataFrame(index=fr_to_tuples.keys(), data=fr_to_tuples.values(), columns=['x1', 'y1', 'x2', 'y2'])

In [234]:
coordinates = pd.DataFrame(index=fr_to_tuples.keys(), data=fr_to_tuples.values(), columns=['x1', 'y1', 'x2', 'y2'])

result = ac_lines.copy()
result = result.join(coordinates)

result.head()

,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km),x1,y1,x2,y2
1,2,1,2024,2,ACTransmissionLine,Pedralva (PT),Sobrado (PT),AC,67,-8.326263,41.594905,-7.046356,42.375793
2,4,1,2022,3,ACTransmissionLine,V.Minho (by Ribeira de Pena),Feira (by Ribeira de Pena),AC,131,-5.738983,43.205176,-8.046112,38.054580
4,18,4,2022,3,ACTransmissionLine,Beariz (ES),Fontefria (ES),AC,30,-8.242493,42.375793,-8.393555,42.174600
5,496,4,2022,3,ACTransmissionLine,Fontefria (ES),Vila Nova de Famalicão (PT) (By Ponte de Lima),AC,140.21,-8.393555,42.174600,-5.766449,36.843362
10,60,23,2022,4,ACTransmissionLine,Avelin/Mastaing (FR),Horta (BE),AC,80,3.353577,50.316531,-3.863068,40.592057


# Determine coordinates using geopy

In [94]:
locator = Nominatim(user_agent='esm_group')
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.05)

In [95]:
x1 = []
y1 = []
x2 = []
y2 = []

error_rows = []

for index, row in ac_lines.iterrows():
    fr   = row['Substation From']
    to   = row['Substation To']
    dist = row['Total route length (km)']

    fr_loc = geocode(fr)
    to_loc = geocode(to)

    if fr_loc is None or to_loc is None:
        error_rows.append([row.values])
        continue
        
    fr_coords  = fr_loc.latitude, fr_loc.longitude
    to_coords  = to_loc.latitude, to_loc.longitude
    coord_dist = distance.distance(fr_coords, to_coords).km

    if math.isclose(coord_dist, dist, rel_tol=0.25):
        x1.append(fr_coords[0])
        y1.append(fr_coords[1])
        
        x2.append(to_coords[0])
        y2.append(to_coords[1])
    else:
        error_rows.append([row.values])
        # x1.append(np.NAN)
        # y1.append(np.NAN)
        
        # x2.append(np.NAN)
        # y2.append(np.NAN)

In [99]:
len(x1), len(error_rows)

(55, 122)

In [92]:
error_rows

[array([4, 1, '2022', 3, 'ACTransmissionLine',
        'V.Minho (by Ribeira de Pena)', 'Feira (by Ribeira de Pena)', 'AC',
        131, nan, nan, nan, nan, nan], dtype=object),
 array([496, 4, '2022', 3, 'ACTransmissionLine', 'Fontefria (ES)',
        'Vila Nova de Famalicão (PT) (By Ponte de Lima)', 'AC', 140.21,
        nan, nan, nan, nan, nan], dtype=object),
 array([60, 23, '2022', 4, 'ACTransmissionLine', 'Avelin/Mastaing (FR)',
        'Horta (BE)', 'AC', 80, nan, nan, nan, nan, nan], dtype=object)]